Import libraries

In [258]:
import yfinance as yf
from datetime import date
import datetime
import pandas as pd

**Downloding the data**

In [259]:
# list of all stock
list_of_stocks = ["SPCE", 
"CPRX", 
"BABA", 
"AMZN", 
"SNE", 
"APHA", 
"TCEHY", 
"GOOGL", 
"MSFT", 
"NRZ", 
"PLUG", 
"PTON", 
"NKE", 
"ATVI", 
"PYPL", 
"FB", 
"GM", 
"KO", 
"V", 
"UBER", 
"MRNA", 
"ZNGA", 
"TXMD", 
"JNJ", 
"RCL", 
"NTDOY", 
"WMT", 
"DKNG", 
"JPM", 
"PENN", 
"SNAP", 
"GE", 
"ET", 
"NOK", 
"BP", 
"DAL", 
"LUV", 
"DIS", 
"SIRI", 
"NFLX", 
"LYFT", 
"NVDA", 
"BAC", 
"AAPL", 
"NIO", 
"WORK", 
"PFE", 
"SQ", 
"SBUX", 
"ZM", 
"KOS", 
"PLAY", 
"GILD", 
"UAL", 
"SAVE", 
"AMD", 
"BA", 
"NCLH", 
"HAL", 
"INTC", 
"T", 
"JBLU", 
"WFC", 
"MRO", 
"INO", 
"RKT", 
"TSLA", 
"CRON", 
"TWTR", 
"CGC", 
"BYND", 
"MGM", 
"AAL", 
"F", 
"ACB", 
"GPRO", 
"MFA", 
"TLRY", 
"CCL", 
"XOM", 
"HEXO", 
"FIT", 
"NKLA", 
"AMC", 
"WKHS", 
"VOO", 
"SPY", 
"GUSH", 
"IVR", 
"USO", 
"UCO", 
"PSEC", 
"FCEL", 
"SPHD", 
"PLTR", 
"IBIO", 
"SRNE", 
"KODK", 
"VTI",
"HOME",
"SOLO",
"BLNK",
"PLUG",
"BLDP"
]

ev_renewable = ["SOLO",
"BLNK",
"PLUG",
"BLDP",
"FCEL"
]

renewable = [
    "EOSE",
    "BLDP",
    "PLUG",
    "BE",
    "FCEL",
    "SBE"
]

top_performing = ["XPEV",
"CELH",
"LI",
"FPRX",
"FOSL",
"PDD",
"FLGT",
"GRWG",
"ATRA",
"NAK",
"DSKE",
"SVC",
"PEYUF",
"BEAM",
"MPLN",
"VRM",
"ENR",
"SFL",
"CCMP",
"GOCO"]

We want to extract two things from data:
1. year-to-date change
2. march 23 crash to date change
3. Aug 31st tech stagnation to date change

In [ ]:
def date_difference(date_1, date_2):
    date_to_date = lambda date: ([date.split('-')[0], date.split('-')[1], date.split('-')[2]])
    
    day_1 = date_to_date(date_1)
    day_2 = date_to_date(date_2)

    date_diff = (date(int(day_2[0]), int(day_2[1]), int(day_2[2])) - date(int(day_1[0]), int(day_1[1]), int(day_1[2]))).days
    return date_diff

def trend_finder(date_1, date_2):
    trend = ((data.loc[date_2]["Close"] - data.loc[date_1]["Close"]) / data.loc[date_1]["Close"]) * (100/date_difference(date_1, date_2))
    return trend

This module gives back the performance over the past days_back days

In [46]:
def day_by_day_record(days_back, interval):
    trends_dict = {}
    
    for i in range(days_back):
        day_1 = (date.today()-datetime.timedelta(i+(2*interval))).strftime("%Y-%m-%d")
        day_2 = (date.today()-datetime.timedelta(i+interval)).strftime("%Y-%m-%d")

        try:
            day_1_to_day_2 = trend_finder(day_1, day_2)
            day_name = 'date ' + str(i+1) + ' to date ' + str(i+2)
            trends_dict[day_name] = day_1_to_day_2
        except:
            continue
            
    trends_df = pd.DataFrame(trends_dict) 
    
    trends_df['average'] = trends_df.mean(axis=1)
    trends_df['standard deviation'] = trends_df.std(axis=1)
    trends_df['normal average'] = trends_df['average'] / trends_df['standard deviation']  
    
    return (trends_df)

Downloading the data 

In [25]:
year_beginning = '2020-01-02'

crash = '2020-03-23'

stagnation = '2020-08-31'

election = '2020-11-03'

today = (date.today()-datetime.timedelta(1)).strftime("%Y-%m-%d")

today = '2020-11-13'


year_beginning_to_crash_trend = trend_finder(year_beginning, crash)
crash_to_stagnation_trend = trend_finder(crash, stagnation)
stagnation_to_election_trend = trend_finder(stagnation, election)
election_to_date_trend = trend_finder(election, today)

trends_dict = { 'year_beginning_to_crash_trend': year_beginning_to_crash_trend,
               'crash_to_stagnation_trend': crash_to_stagnation_trend, 
         'stagnation_to_election_trend': stagnation_to_election_trend,
        'election_to_date_trend': election_to_date_trend} 

In [ ]:
# Returns the current local date 
today = date.today() 

data = yf.download(renewable,'2020-01-01',today)

In [267]:
# downloading data from Yahoo! Finance)
data = yf.download(list_of_stocks,'2020-01-01', date.today())

[*********************100%***********************]  103 of 103 completed


In [322]:
def day_by_day_record(df, count_back, interval, date = date.today()):
    
    # getting all active stock dates from the downloaded data
    stock_active_dates = [str(date).split('T')[0] for date in list(df.index.values)]
    
    T = 1
    
    # finding the first valid date

    while True:
        start_date = (date-datetime.timedelta(T)).strftime("%Y-%m-%d")
        
        if start_date in stock_active_dates: 
            start_date = date-datetime.timedelta(T)
            break
        
        else:
            T += 1
    
    # starting from start date and going back in time until we find enough valid dates
            
    valid_dates = [(date-datetime.timedelta(T)).strftime("%Y-%m-%d")]    
    
    i = 1

    while len(valid_dates) < count_back:
        
        # this day is closer to today
        test_day = (start_date - datetime.timedelta(i*interval+1))

        if test_day.strftime("%Y-%m-%d") in stock_active_dates:
            valid_dates.append(test_day.strftime("%Y-%m-%d"))
            
        # fidning the closest day that works 
        else:
            T = 1
            
            while True:
                approx_date = (test_day - datetime.timedelta(T)).strftime("%Y-%m-%d")

                if approx_date in stock_active_dates: 
                    valid_dates.append(approx_date)
                    break

                else:
                    T += 1
                    
        i += 1
        
    trends_dict = {}
    for i in range(len(valid_dates)):
        if i < len(valid_dates)-1:
            date_1 = valid_dates[i]
            date_2 = valid_dates[i+1]
        
        try:
            trend = ((df.loc[date_2]["Close"] - df.loc[date_1]["Close"]) / df.loc[date_1]["Close"]) * (100/date_difference(date_1, date_2))

            day_name = str(date_1) + '   ' + str(date_2)
            trends_dict[day_name] = trend
        except:
            continue

    trends_df = pd.DataFrame(trends_dict) 

    trends_df['average'] = trends_df.mean(axis=1)
    trends_df['standard deviation'] = trends_df.std(axis=1)
    trends_df['normal average'] = trends_df['average'] / (trends_df['standard deviation']**2)  

    return (trends_df)

In [323]:
# day_by_day_record(df, count_back, interval, start date*default at today*)

trends_dataframe_90 = day_by_day_record(data, 90, 1, date = date.today()- datetime.timedelta(60))

trends_dataframe_60 = day_by_day_record(data, 60, 1, date = date.today()- datetime.timedelta(30))

trends_dataframe_30 = day_by_day_record(data, 30, 1)

trends_dataframe_7 = day_by_day_record(data, 7, 1)

In [335]:
c = trends_dataframe_90.sort_values(by=['average'], ascending=False)

A = c[c['average'] > 0].index

In [336]:
c = trends_dataframe_60.sort_values(by=['average'], ascending=False)

B = c[c['average'] > 0].index

In [337]:
c = trends_dataframe_30.sort_values(by=['average'], ascending=False)

C = c[c['average'] > 0].index

In [341]:
c = trends_dataframe_7.sort_values(by=['average'], ascending=False)

D = c[c['average'] > 0].index

D

Index(['NIO', 'BABA', 'INTC', 'TCEHY', 'SNAP', 'NFLX', 'TSLA', 'SNE', 'SBUX',
       'V', 'PLAY', 'JNJ', 'UCO', 'GILD', 'RCL', 'AMZN', 'KO', 'DIS', 'ATVI',
       'SIRI', 'JPM', 'USO', 'IVR', 'NVDA', 'NOK', 'LUV', 'NCLH', 'SPY', 'VOO',
       'GE', 'NTDOY', 'BAC', 'WFC', 'CCL', 'T', 'DAL', 'FIT', 'FB', 'VTI',
       'SPHD'],
      dtype='object')

In [339]:
u = set.intersection(set(A),set(B),set(C), set(D))
u

{'CCL',
 'DAL',
 'GE',
 'JPM',
 'KO',
 'NCLH',
 'NIO',
 'NTDOY',
 'PLAY',
 'SBUX',
 'SNAP',
 'SNE',
 'TCEHY',
 'TSLA',
 'VTI'}